In [ ]:
%%capture
pip install mne

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mne
import pywt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'

In [ ]:
epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']
epochs_path

['/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN0-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN1-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN2-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN3-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN4-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN5-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN6-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN7-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN8-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN9-epo.fif',
 '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN10-epo.fif',
 '/conten

In [ ]:
len(epochs_path)

39

In [ ]:
epochs = mne.read_epochs(epochs_path[26]).pick_types(eeg=True)

Reading /content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch/PN33-epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [ ]:
epochs.get_data().shape

<ipython-input-8-79b916371608>:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs.get_data().shape


(50, 29, 180)

In [ ]:
# epochs.events[:,2]

### Total data stacked


In [ ]:
%%capture
data= [mne.read_epochs(i).pick_types(eeg=True) for i in epochs_path]
labels = [mne.read_epochs(i).events[:,2] for i in epochs_path]

In [ ]:
# for i,j in enumerate(epochs_path[12:]):
#     epoch = mne.read_epochs(j).pick_types(eeg=True).equalize_event_counts()
#     epoch[0].save(f'E:/Data WareHouse/1sec/Epileptic Seizure/equalized/PN{i+19}-epo.fif', overwrite= True)


In [ ]:
len(data),len(labels)

(39, 39)

In [ ]:
group = [[i]*len(j) for i,j in enumerate(data)]

In [ ]:
# for i in range(0,36):
#     print(data[i].shape)

In [ ]:
# X=np.vstack(data)
Y=np.hstack(labels)
group= np.hstack(group)
# print(X.shape,Y.shape,group.shape)

In [ ]:
# np.savez("E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/X_Y_G_for1secdata.npz",X = X,Y = Y,G = group)

#feature extraction
Without DWT. feature extarction on the processed data.

#Time Domain

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

def extract_time_domain_features(epochs):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Flatten the channel data
            flattened_data = channel_data.flatten()

            # Basic Time-Domain Features
            mean_val = np.mean(flattened_data)
            median_val = np.median(flattened_data)
            var_val = np.var(flattened_data)
            std_dev = np.std(flattened_data)
            skewness = skew(flattened_data)
            kurt = kurtosis(flattened_data)
            zcr = zero_crossing_rate(flattened_data)
            peak_amp = np.ptp(flattened_data)

            # Hjorth Parameters
            activity, mobility, complexity = hjorth_parameters(flattened_data)

            # Additional Features
            num_waves = len(find_peaks(flattened_data)[0])
            wave_duration = len(flattened_data) / num_waves if num_waves > 0 else 0

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
                wave_duration, peak_amp, activity, mobility, complexity
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

##Frequency Domain

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch



def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


def extract_frequency_domain_features(epochs, sfreq,wavelet='db4', bands={'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100), 'sigma': (11, 16)}):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Compute the Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, sfreq, nperseg=256)

            # Frequency domain features
            mean_val = np.mean(psd)
            median_val = np.median(psd)
            var_val = np.var(psd)
            std_dev = np.std(psd)
            skewness = skew(psd)
            kurt = kurtosis(psd)

            # Compute wavelet coefficients
            wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
            wave_coeffs_mean = np.mean(wave_coeffs[0])

            # Band Power Features
            band_powers = {}
            for band, freq_range in bands.items():
                freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
                band_power = np.sum(psd[freq_mask])
                band_powers[band] = band_power

            # Band Power Ratios
            theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
            beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
            theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']
            # Additional Band Power Ratios
            theta_beta_ratio = band_powers['theta'] / band_powers['beta']
            theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
            gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
            gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])


            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt,
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
                theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,theta_beta_ratio,
                theta_alpha_beta_alpha_ratio, gamma_delta_ratio, gamma_beta_delta_alpha_ratio,
                wave_coeffs_mean
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

#Band Separation

In [ ]:
%%capture
#####Delta epoch
delta_epo= [i.copy().filter(1.59,4) for i in data]
#####Theta epoch
theta_epo= [i.copy().filter(4,8) for i in data]
#####alpha epoch
alpha_epo= [i.copy().filter(8,13) for i in data]
#####Delta epoch
beta_epo= [i.copy().filter(13,30) for i in data]

In [ ]:
#stack the epochs
delta_stack = np.vstack([i.get_data() for i in delta_epo])
theta_stack = np.vstack([i.get_data() for i in theta_epo])
alpha_stack = np.vstack([i.get_data() for i in alpha_epo])
beta_stack = np.vstack([i.get_data() for i in beta_epo])

<ipython-input-18-f2a9e2b3c95f>:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  delta_stack = np.vstack([i.get_data() for i in delta_epo])
<ipython-input-18-f2a9e2b3c95f>:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  delta_stack = np.vstack([i.get_data() for i in delta_epo])
<ipython-input-18-f2a9e2b3c95f>:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  delta_stack = np.vstack([i.get_data() for i in delta_epo])
<ipython-input-18-f2a9e2b3c95f>:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  delta_stack = np.vstack([i.get_data() for i in delta_epo])
<ipython-input-18-f2a9e2b3c95f>:2: FutureWarning: The current defaul

#Apply Feature Extraction


##Time domain

In [ ]:
# Extract features from epochs
  # Assuming epochs is a list of MNE Epochs objects
delta_time_domain_features = extract_time_domain_features(delta_stack)
theta_time_domain_features = extract_time_domain_features(theta_stack)
alpha_time_domain_features = extract_time_domain_features(alpha_stack)
beta_time_domain_features = extract_time_domain_features(beta_stack)

In [ ]:

# all_features.shape
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/delta_time_domain.npy",delta_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/theta_time_domain.npy",theta_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/alpha_time_domain.npy",alpha_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /RAW/beta_time_domain.npy",beta_time_domain_features)




##Frequency Domain

In [ ]:
# Example usage
sfreq = 256  # Replace with the sampling frequency of your data
# epochs_data = [epoch.get_data() for epoch in epochs]  # Assuming epochs is a list of MNE Epochs objects
delta_frequency_features = extract_frequency_domain_features(delta_stack, sfreq)
theta_frequency_features = extract_frequency_domain_features(theta_stack, sfreq)
alpha_frequency_features = extract_frequency_domain_features(alpha_stack, sfreq)
beta_frequency_features = extract_frequency_domain_features(beta_stack, sfreq)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 180, using nperseg = 180
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:

np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/RAW/delta_frequency_domain.npy",delta_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/RAW/theta_frequency_domain.npy",theta_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/RAW/alpha_frequency_domain.npy",alpha_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/RAW/beta_frequency_domain.npy",beta_frequency_features)

##Merge time and frequency domain

In [ ]:
# Assuming all_features and frequency_features are your numpy arrays
delta_merged_features = np.concatenate((delta_time_domain_features ,delta_frequency_features), axis=2)
theta_merged_features = np.concatenate((theta_time_domain_features ,theta_frequency_features), axis=2)
alpha_merged_features = np.concatenate((alpha_time_domain_features ,alpha_frequency_features), axis=2)
beta_merged_features = np.concatenate((beta_time_domain_features ,beta_frequency_features), axis=2)

In [ ]:
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time feature domain/RAW/delta_MERGE_domain.npy",delta_merged_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time feature domain/RAW/theta_MERGE_domain.npy",theta_merged_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time feature domain/RAW/alpha_MERGE_domain.npy",alpha_merged_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time feature domain/RAW/beta_MERGE_domain.npy",beta_merged_features)

# DWT
dwt is a signal processing method


### Devide the respected epochs in bands than get the DWT

In [ ]:
data[0].info

<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, F3, C3, P3, O1, F7, T3, T5, FC1, FC5, CP1, CP5, F9, Fz, Cz, ...
 chs: 29 EEG
 custom_ref_applied: True
 dig: 32 items (3 Cardinal, 29 EEG)
 file_id: 4 items (dict)
 highpass: 1.6 Hz
 lowpass: 30.0 Hz
 meas_date: 2016-01-01 19:39:33 UTC
 meas_id: 4 items (dict)
 nchan: 29
 projs: []
 sfreq: 256.0 Hz
>

In [ ]:
delta_epo[0].info

<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, F3, C3, P3, O1, F7, T3, T5, FC1, FC5, CP1, CP5, F9, Fz, Cz, ...
 chs: 29 EEG
 custom_ref_applied: True
 dig: 32 items (3 Cardinal, 29 EEG)
 file_id: 4 items (dict)
 highpass: 1.6 Hz
 lowpass: 4.0 Hz
 meas_date: 2016-01-01 19:39:33 UTC
 meas_id: 4 items (dict)
 nchan: 29
 projs: []
 sfreq: 256.0 Hz
>

In [ ]:
theta_epo[0].info

<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, F3, C3, P3, O1, F7, T3, T5, FC1, FC5, CP1, CP5, F9, Fz, Cz, ...
 chs: 29 EEG
 custom_ref_applied: True
 dig: 32 items (3 Cardinal, 29 EEG)
 file_id: 4 items (dict)
 highpass: 4.0 Hz
 lowpass: 8.0 Hz
 meas_date: 2016-01-01 19:39:33 UTC
 meas_id: 4 items (dict)
 nchan: 29
 projs: []
 sfreq: 256.0 Hz
>

In [ ]:
alpha_epo[0].info

<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, F3, C3, P3, O1, F7, T3, T5, FC1, FC5, CP1, CP5, F9, Fz, Cz, ...
 chs: 29 EEG
 custom_ref_applied: True
 dig: 32 items (3 Cardinal, 29 EEG)
 file_id: 4 items (dict)
 highpass: 8.0 Hz
 lowpass: 13.0 Hz
 meas_date: 2016-01-01 19:39:33 UTC
 meas_id: 4 items (dict)
 nchan: 29
 projs: []
 sfreq: 256.0 Hz
>

In [ ]:
beta_epo[0].info

<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, F3, C3, P3, O1, F7, T3, T5, FC1, FC5, CP1, CP5, F9, Fz, Cz, ...
 chs: 29 EEG
 custom_ref_applied: True
 dig: 32 items (3 Cardinal, 29 EEG)
 file_id: 4 items (dict)
 highpass: 13.0 Hz
 lowpass: 30.0 Hz
 meas_date: 2016-01-01 19:39:33 UTC
 meas_id: 4 items (dict)
 nchan: 29
 projs: []
 sfreq: 256.0 Hz
>

In [ ]:
# delta_epo.save('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/delta_epo.fif', overwrite=True)
# alpha_epo.save('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/alpha_epo.fif', overwrite=True)
# beta_epo.save('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/beta_epo.fif', overwrite=True)
# theta_epo.save('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/theta_epo.fif', overwrite=True)

In [ ]:
# eeg_data = np.load('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/X_Y_G_for1secdata.npz')


In [ ]:
import pywt

def extract_dwt_features(epochs, wavelet='db4', level=4):
    """
    Extract DWT features from EEG epochs.

    Parameters:
    epochs: 3D numpy array, shape (n_epochs, n_channels, n_samples)
    wavelet: str, type of mother wavelet to use
    level: int, level of decomposition

    Returns:
    dwt_features: 3D numpy array of DWT features
    """

    n_epochs, n_channels, _ = epochs.get_data().shape
    dwt_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            # Perform DWT
            coeffs = pywt.wavedec(channel, wavelet, level=level)
            # Concatenate DWT coefficients
            concatenated_coeffs = np.concatenate([c for c in coeffs], axis=0)
            epoch_features.append(concatenated_coeffs)
        dwt_features.append(epoch_features)

    return np.array(dwt_features)

In [ ]:
%%capture
# Example usage
# Assuming 'epochs' is your EEG data with shape (n_epochs, n_channels, n_samples)
delta_dwt_features = [extract_dwt_features(i) for i in delta_epo]
theta_dwt_features = [extract_dwt_features(i) for i in theta_epo]
alpha_dwt_features = [extract_dwt_features(i) for i in alpha_epo]
beta_dwt_features = [extract_dwt_features(i) for i in beta_epo]

delta_dwt_features_stack = np.vstack(delta_dwt_features)
theta_dwt_features_stack = np.vstack(theta_dwt_features)
alpha_dwt_features_stack = np.vstack(alpha_dwt_features)
beta_dwt_features_stack = np.vstack(beta_dwt_features)

In [ ]:
# beta_dwt_features_stack.shape

In [ ]:
# np.save("E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/DWT Feature extraction.npy", dwt_features)

In [ ]:
# dwt_features = np.load('E:/Data WareHouse/Epileptic Seizure/Feature extraction/DWT Feature extraction.npy')

In [ ]:
# dwt_features.shape

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# # Assuming dwt_features is the output from your extract_dwt_features function
# # Reshape dwt_features to 2D for PCA and normalization
# nsamples, nx, ny = dwt_features.shape
# dwt_features_2d = dwt_features.reshape((nsamples, nx*ny))



# # Normalize the features
# scaler = StandardScaler()
# dwt_features_normalized = scaler.fit_transform(dwt_features_2d)

# # Apply PCA for dimensionality reduction
# pca = PCA(n_components=0.95)  # Retain 95% of variance
# dwt_features_pca = pca.fit_transform(dwt_features_normalized)

In [ ]:
# dwt_features_pca.shape

# Equalize the data


In [ ]:
# epochs = [mne.read_epochs(i).pick_types(eeg=True).equalize_event_counts() for i in epochs_path]
# # labels = [i.events[:,2] for i in data]

In [ ]:
# len(epochs)

In [ ]:
# equ_data = []
# equ_labels = []
# def equalizerFun(epoch, equ_data,equ_labels ):
#     epoch.equalize_event_counts()
#     equalize_data = epoch.get_data()
#     equalize_labels = epoch.events[:,2]
#     equ_data.append(equalize_data)
#     equ_labels.append(equalize_labels)
#     del epoch,equalize_labels,equ_data

#     equalizerFun(i,equ_data,equ_labels)

In [ ]:
# # data[0].equalize_event_counts
# equalize_epoch_count  = [i.equalize_event_counts() for i in epochs]

In [ ]:
# equalize_data = [equalize_epoch_count[i][0]for i in range(0,36)]

In [ ]:
# equalize_data.

In [ ]:
# equalize_data = [equalize_epoch_count[i][0].get_data() for i in range(0,36)]
# equalize_labels = [equalize_epoch_count[i][0].events[:,2] for i in range(0,36)]

In [ ]:
# len(equalize_data),len(equalize_labels)

In [ ]:
# Eq_group=[[i]*len(j) for i,j in enumerate(equalize_data) ]

In [ ]:
# Eq_X=np.vstack(equalize_data)
# Eq_Y=np.hstack(equalize_labels)
# Eq_group= np.hstack(Eq_group)
# print(Eq_X.shape,Eq_Y.shape,Eq_group.shape)

In [ ]:
# np.savez("E:/Data WareHouse/Epileptic Seizure/balanced epochs/run_X_Y_group.npz",Eq_X,Eq_Y,Eq_group)

In [ ]:
# eq_data_DL = np.moveaxis(Eq_X,1,2) #moving the axis of channels

In [ ]:
# eq_data_DL.shape

# Feature Extraction

In [ ]:
# Eq_file_path = 'E:/Data WareHouse/Epileptic Seizure/balanced epochs/run_X_Y_group.npz'
# eq_file = np.load(Eq_file_path)
# X = eq_file['arr_0']
# Y = eq_file['arr_1']
# G = eq_file['arr_2']

In [ ]:
# X.shape

## Time Domain Features

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

def extract_time_domain_features(epochs):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Flatten the channel data
            flattened_data = channel_data.flatten()

            # Basic Time-Domain Features
            mean_val = np.mean(flattened_data)
            median_val = np.median(flattened_data)
            var_val = np.var(flattened_data)
            std_dev = np.std(flattened_data)
            skewness = skew(flattened_data)
            kurt = kurtosis(flattened_data)
            zcr = zero_crossing_rate(flattened_data)
            peak_amp = np.ptp(flattened_data)

            # Hjorth Parameters
            activity, mobility, complexity = hjorth_parameters(flattened_data)

            # Additional Features
            num_waves = len(find_peaks(flattened_data)[0])
            wave_duration = len(flattened_data) / num_waves if num_waves > 0 else 0

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
                wave_duration, peak_amp, activity, mobility, complexity
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)




In [ ]:
# Extract features from epochs
  # Assuming epochs is a list of MNE Epochs objects
delta_time_domain_features = extract_time_domain_features(delta_dwt_features_stack)
theta_time_domain_features = extract_time_domain_features(theta_dwt_features_stack)
alpha_time_domain_features = extract_time_domain_features(alpha_dwt_features_stack)
beta_time_domain_features = extract_time_domain_features(beta_dwt_features_stack)

In [ ]:
# all_features.shape
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /delta_time_domain.npy",delta_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /theta_time_domain.npy",theta_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /alpha_time_domain.npy",alpha_time_domain_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time domain /beta_time_domain.npy",beta_time_domain_features)



In [ ]:
# np.savez('E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/time_domaindata.npz', X=all_features, Y= Y, G= group)

In [ ]:
# Replace with your array
# np.save('E:/Data WareHouse/Epileptic Seizure/Feature extraction/equalized_data_timedomainfeatures.npy', all_features)

##  Frequency-domain features

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch



def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


def extract_frequency_domain_features(epochs, sfreq,wavelet='db4', bands={'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100), 'sigma': (11, 16)}):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Compute the Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, sfreq, nperseg=256)

            # Frequency domain features
            mean_val = np.mean(psd)
            median_val = np.median(psd)
            var_val = np.var(psd)
            std_dev = np.std(psd)
            skewness = skew(psd)
            kurt = kurtosis(psd)

            # Compute wavelet coefficients
            wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
            wave_coeffs_mean = np.mean(wave_coeffs[0])

            # Band Power Features
            band_powers = {}
            for band, freq_range in bands.items():
                freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
                band_power = np.sum(psd[freq_mask])
                band_powers[band] = band_power

            # Band Power Ratios
            theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
            beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
            theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']
            # Additional Band Power Ratios
            theta_beta_ratio = band_powers['theta'] / band_powers['beta']
            theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
            gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
            gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])


            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt,
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
                theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,theta_beta_ratio,
                theta_alpha_beta_alpha_ratio, gamma_delta_ratio, gamma_beta_delta_alpha_ratio,
                wave_coeffs_mean
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)




In [ ]:
# Example usage
sfreq = 256  # Replace with the sampling frequency of your data
# epochs_data = [epoch.get_data() for epoch in epochs]  # Assuming epochs is a list of MNE Epochs objects
delta_frequency_features = extract_frequency_domain_features(delta_dwt_features_stack, sfreq)
theta_frequency_features = extract_frequency_domain_features(theta_dwt_features_stack, sfreq)
alpha_frequency_features = extract_frequency_domain_features(alpha_dwt_features_stack, sfreq)
beta_frequency_features = extract_frequency_domain_features(beta_dwt_features_stack, sfreq)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 205, using nperseg = 205
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/delta_frequency_domain.npy",delta_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/theta_frequency_domain.npy",theta_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/alpha_frequency_domain.npy",alpha_frequency_features)
np.save("/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/feature domain/beta_frequency_domain.npy",beta_frequency_features)

In [ ]:
# # Replace with your array
# np.savez('E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/feature_domaindata.npz', X = frequency_features, Y=Y, G= group)

In [ ]:
# frequency_features.shape

(5834, 29, 20)

## DWT

In [ ]:
eeg_data = np.load('E:/Data WareHouse/1sec/Epileptic Seizure/equalized/equalized data/X_Y_G_for1secdata.npz')


In [ ]:
import pywt
import numpy as np

def extract_dwt_features(epochs, wavelet='db4', level=4):
    """
    Extract DWT features from EEG epochs.

    Parameters:
    epochs: 3D numpy array, shape (n_epochs, n_channels, n_samples)
    wavelet: str, type of mother wavelet to use
    level: int, level of decomposition

    Returns:
    dwt_features: 3D numpy array of DWT features
    """

    n_epochs, n_channels, _ = epochs.get_data().shape
    dwt_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            # Perform DWT
            coeffs = pywt.wavedec(channel, wavelet, level=level)
            # Concatenate DWT coefficients
            concatenated_coeffs = np.concatenate([c for c in coeffs], axis=0)
            epoch_features.append(concatenated_coeffs)
        dwt_features.append(epoch_features)

    return np.array(dwt_features)

# Example usage
# Assuming 'epochs' is your EEG data with shape (n_epochs, n_channels, n_samples)
dwt_features = [extract_dwt_features(i) for i in data]


In [ ]:
dwt_features= np.vstack(dwt_features)

In [ ]:
dwt_features.shape

(5834, 29, 205)

In [ ]:
np.save("E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/DWT Feature extraction.npy", dwt_features)

In [ ]:
dwt_features = np.load('E:/Data WareHouse/Epileptic Seizure/Feature extraction/DWT Feature extraction.npy')

In [ ]:
dwt_features.shape

(5834, 29, 205)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming dwt_features is the output from your extract_dwt_features function
# Reshape dwt_features to 2D for PCA and normalization
nsamples, nx, ny = dwt_features.shape
dwt_features_2d = dwt_features.reshape((nsamples, nx*ny))



# Normalize the features
scaler = StandardScaler()
dwt_features_normalized = scaler.fit_transform(dwt_features_2d)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Retain 95% of variance
dwt_features_pca = pca.fit_transform(dwt_features_normalized)

In [ ]:
dwt_features_pca.shape

(5834, 330)

In [ ]:
# RF=RandomForestClassifier()
# ada=AdaBoostClassifier(base_estimator= RandomForestClassifier())
# RFC=Pipeline([('scaler',StandardScaler()),('RF',RF)])
# # ABC=Pipeline([('scaler',StandardScaler()),('ada',ada)])
# # # param_grid = {
# #     'RF__n_estimators': [100],  # Try different values for n_estimators
# #     # 'RF__max_depth': [],  # Try different values for max_depth
# # }

# # model=GridSearchCV(estimator=pipe,param_grid=param_grid, cv=5)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'n_estimators': [1000],  # Try different values for n_estimators
#     # 'criterion': ['entropy'],
#     # 'max_depth' : [7],
#     # 'min_samples_split': np.arange(1,10),
#     # 'RF__max_depth': [],  # Try different values for max_depth
# }
# model=GridSearchCV(estimator=RF,param_grid=param_grid, cv=3)

In [ ]:
# # Random Forest
# model.fit(dwt_features_pca, Y)
# print(model.best_estimator_)
# print(model.best_score_)

RandomForestClassifier(n_estimators=1000)
0.6035329180898801


In [ ]:
dwt_features = np.moveaxis(dwt_features, 2,1)
# dwt_features

In [ ]:
#Train test split data.
dwt_features_pca.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.backend import clear_session

# Assuming dwt_features_pca is your processed input data and Y is your labels
y = Y  # Your target labels
clear_session()
# Define the model
model = Sequential()
model.add(Flatten(input_shape=(dwt_features_pca.shape[1],)))# Flatten the input
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))  # First hidden layer
model.add(Dense(64, activation='relu'))   # Second hidden layer
model.add(Dense(32, activation='relu'))   # Second hidden layer
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='relu'))   # Second hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(dwt_features_pca, Y, epochs=50, batch_size=16, validation_split=0.2,shuffle=True)

Epoch 1/50
292/292 [==============================] - 2s 4ms/step - loss: 0.5562 - accuracy: 0.7064 - val_loss: 0.4688 - val_accuracy: 0.7943
Epoch 2/50
292/292 [==============================] - 1s 3ms/step - loss: 0.3462 - accuracy: 0.8614 - val_loss: 0.4455 - val_accuracy: 0.8012
Epoch 3/50
292/292 [==============================] - 1s 3ms/step - loss: 0.2252 - accuracy: 0.9184 - val_loss: 0.5130 - val_accuracy: 0.7883
Epoch 4/50
292/292 [==============================] - 1s 3ms/step - loss: 0.1711 - accuracy: 0.9402 - val_loss: 0.4678 - val_accuracy: 0.8295
Epoch 5/50
292/292 [==============================] - 1s 3ms/step - loss: 0.1494 - accuracy: 0.9541 - val_loss: 0.6152 - val_accuracy: 0.7909
Epoch 6/50
292/292 [==============================] - 1s 3ms/step - loss: 0.1124 - accuracy: 0.9646 - val_loss: 0.5951 - val_accuracy: 0.8038
Epoch 7/50
292/292 [==============================] - 1s 3ms/step - loss: 0.1136 - accuracy: 0.9676 - val_loss: 0.5912 - val_accuracy: 0.8201
Epoch 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten

# Assuming dwt_features is your DWT data with shape (946, 29, 205)
X = dwt_features_pca  # Your input data
y = Y    # Your target labels

# Define the LSTM model
model = Sequential()
model.add(LSTM(10, activation = 'relu',input_shape=(dwt_features_pca.shape[1], )))  # Adjust the number of units as needed
model.add(LSTM(10))
model.add(Flatten())# Flatten the input
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))  # First hidden layer
model.add(Dense(64, activation='relu'))   # Second hidden layer
model.add(Dense(32, activation='relu'))   # Second hidden layer
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='relu'))# Second hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer  # Use 'softmax' for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class

# Train the model
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)



ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 330)

# Train Test Validation Split

In [ ]:
def data_sep(X,Y):
    # Convert Y to a numpy array if it's not already
    Y = np.array(Y)

    # Separate the features based on the labels
    X_1 = X[Y == 1]
    X_0 = X[Y == 0]

    # Separate the labels based on their values
    Y_1 = Y[Y == 1]
    Y_0 = Y[Y == 0]
    return X_1,X_0,Y_1,Y_0





def train_test_spilt(X_1,X_0,Y_1,Y_0, split=.2):
    '''For the binary data this train test split is used.'''



    X_train = np.concatenate((X_1[round(len(X_1)*split):],X_0[round(len(X_0)*split):]), axis=0)
    X_test = np.concatenate((X_1[:round(len(X_1)*split)],X_0[:round(len(X_0)*split)]), axis=0)

    Y_train = np.concatenate((Y_1[round(len(Y_1)*split):],Y_0[round(len(Y_0)*split):]), axis=0)
    Y_test = np.concatenate((Y_1[:round(len(Y_1)*split)],Y_0[:round(len(Y_0)*split)]), axis=0)

    return X_train,Y_train, X_test, Y_test

In [ ]:
import random

X_1,X_0,Y_1,Y_0 = data_sep(X,Y)

def add_noise(eeg_data, noise_level=0.01):
    """ Add random noise to the EEG data """
    noise = np.random.normal(0, noise_level, eeg_data.shape)
    return eeg_data + noise

def time_shift(eeg_data, shift_max=10):
    """ Randomly shift the EEG data in time """
    shift = random.randint(-shift_max, shift_max)
    return np.roll(eeg_data, shift, axis=1)

def scale_amplitude(eeg_data, scale_factor_range=(0.9, 1.1)):
    """ Randomly scale the amplitude of the EEG data """
    scale_factor = random.uniform(*scale_factor_range)
    return eeg_data * scale_factor

def mirror_eeg(eeg_data):
    """ Mirror the EEG data in time """
    return np.flip(eeg_data, axis=1)

# Example of using the augmentation functions
# Assuming 'eeg_epoch' is a single EEG epoch

eeg_epoch = X_1  # Replace with an actual epoch

augmented_epoch_noise1 = add_noise(eeg_epoch, noise_level=0.01)
augmented_epoch_noise2 = add_noise(eeg_epoch, noise_level=0.02)
augmented_epoch_noise3 = add_noise(eeg_epoch, noise_level=0.03)
augmented_epoch_noise4 = add_noise(eeg_epoch, noise_level=0.04)
augmented_epoch_shifted = time_shift(eeg_epoch)
augmented_epoch_scaled = scale_amplitude(eeg_epoch)
augmented_epoch_mirrored = mirror_eeg(eeg_epoch)

In [ ]:
# Concatenate all augmented data
all_augmented_data = np.concatenate([
                                    X_1,
                                    augmented_epoch_noise1,
    augmented_epoch_noise2,
    augmented_epoch_noise3,
    augmented_epoch_noise4,
                                     augmented_epoch_shifted,
                                     augmented_epoch_scaled,
                                     augmented_epoch_mirrored], axis=0)

# Create labels, each epoch labeled as 1
labels = np.ones((all_augmented_data.shape[0],), dtype=int)

In [ ]:
all_augmented_data.shape

(3784, 29, 180)

In [ ]:
labels.shape

(3784,)

In [ ]:
X_train,Y_train, X_test, Y_test = train_test_spilt(all_augmented_data,X_0,labels,Y_0)

In [ ]:
X_train.shape,X_test.shape,Y_test.shape, Y_train.shape

((19773, 29, 180), (4944, 29, 180), (4944,), (19773,))

# Imbalanced data

## Machine Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, LeakyReLU,\
                                    UpSampling1D,Reshape
from tensorflow.keras.backend import clear_session

In [ ]:
# Path to the file
file_path1 = 'E:/Data WareHouse/Epileptic Seizure/Feature extraction/timedomainfeature.npy'
file_path2 = 'E:/Data WareHouse/Epileptic Seizure/Feature extraction/frequencydomain.npy'
# Load the array
time_domain_features = np.load(file_path1)
frequencydomain = np.load(file_path2)

In [ ]:
import numpy as np

# Assuming all_features and frequency_features are your numpy arrays
merged_features = np.concatenate((time_domain_features, frequencydomain), axis=2)

# Checking the new shape
print(merged_features.shape)

(21406, 29, 33)


In [ ]:
Y.shape

(21406,)

In [ ]:
#Change the shape of the array
# Reshape the data to 2D
eeg_data_reshaped = merged_features.reshape(merged_features.shape[0], -1)

In [ ]:
eeg_data_reshaped.shape, Y.shape

((21406, 957), (21406,))

In [ ]:
X1,X0,Y1,Y0 = data_sep(eeg_data_reshaped,Y)

In [ ]:
X_train,Y_train, X_test, Y_test = train_test_spilt(X1,X0,Y1,Y0,split=.3)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((14984, 957), (6422, 957), (14984,), (6422,))

In [ ]:
X_train[Y_train==1].shape,X_test[Y_test==1].shape

((331, 957), (142, 957))

In [ ]:
X_train[Y_train==0].shape,X_test[Y_test==0].shape

((14653, 957), (6280, 957))

In [ ]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

In [ ]:
RF=RandomForestClassifier()
ada=AdaBoostClassifier(base_estimator= RandomForestClassifier())
RFC=Pipeline([('scaler',StandardScaler()),('RF',RF)])
ABC=Pipeline([('scaler',StandardScaler()),('ada',ada)])
# param_grid = {
#     'RF__n_estimators': [100],  # Try different values for n_estimators
#     # 'RF__max_depth': [],  # Try different values for max_depth
# }

# model=GridSearchCV(estimator=pipe,param_grid=param_grid, cv=5)

In [ ]:
# Random Forest
model.fit(X_train, Y_train)

pred = model.predict(X_test)

accuracy_score(Y_test, pred)

0.981314232326378

In [ ]:
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

Precision: 0.96
Recall: 0.16
F1 Score: 0.28
Sensitivity: 0.16
Specificity: 1.00


In [ ]:
# AdaBoost
# Random Forest
ada.fit(X_train, Y_train)

pred = ada.predict(X_test)

print(accuracy_score(Y_test, pred))
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.9810028028651511
Precision: 1.00
Recall: 0.14
F1 Score: 0.25
Sensitivity: 0.14
Specificity: 1.00


In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                       random_state=0)

BC=Pipeline([('scaler',StandardScaler()),('bc',bc)])
BC.fit(X_train, Y_train) #doctest:


pred = BC.predict(X_test)
print('Accuracy: ',balanced_accuracy_score(Y_test,pred))
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy:  0.6046783888041626
Precision: 0.71
Recall: 0.21
F1 Score: 0.33
Sensitivity: 0.21
Specificity: 1.00


In [ ]:
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)
SVM=Pipeline([('scaler',StandardScaler()),('svm',svc_model)])


SVM.fit(X_train, Y_train)



pred = SVM.predict(X_test)
print('Accuracy: ',balanced_accuracy_score(Y_test,pred))
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

Accuracy:  0.5809859154929577
Precision: 1.00
Recall: 0.16
F1 Score: 0.28
Sensitivity: 0.16
Specificity: 1.00


In [ ]:
from sklearn.preprocessing import StandardScaler



def scale_3d_data(data):
    # Assuming X is your 3D data with shape (samples, channels, features)
    samples, channels, features = data.shape

    # Reshape X to 2D for scaling - shape becomes (samples * channels, features)
    X_reshaped = data.reshape(-1, features)

    # Initialize the scaler
    scaler = StandardScaler()

    # Fit and transform the data
    X_scaled_reshaped = scaler.fit_transform(X_reshaped)

    # Reshape the scaled data back to 3D - shape becomes (samples, channels, features)
    X_scaled = X_scaled_reshaped.reshape(samples, channels, features)
    return X_scaled


X_train_Scale= scale_3d_data(X_train)

X_test_Scale= scale_3d_data(X_test)
X_train_Scale.shape,X_test_Scale.shape

((19773, 29, 180), (4944, 29, 180))

In [ ]:


# def cnnmodel(input_shape):
#     clear_session()
#     model = Sequential()

#     # Adjusted kernel sizes and strides
#     model.add(Conv1D(filters=256, kernel_size=3, input_shape=input_shape))  # First convolutional layer
#     model.add(LeakyReLU())
#     model.add(MaxPool1D(pool_size=2, strides=2))
#     model.add(Dropout(0.5))

#     model.add(Conv1D(filters=128, kernel_size=3, strides=1))  # Second convolutional layer
#     model.add(LeakyReLU())
#     model.add(MaxPool1D(pool_size=2, strides=2))
#     model.add(Dropout(0.5))

#     model.add(Conv1D(filters=64, kernel_size=3, strides=1))  # Third convolutional layer
#     model.add(LeakyReLU())
#     # Maybe remove this pooling layer if the size reduces too much
#     # model.add(MaxPool1D(pool_size=2, strides=2))
#     model.add(Dropout(0.5))

#     model.add(Conv1D(filters=32, kernel_size=3, strides=1))  # Fourth convolutional layer
#     model.add(LeakyReLU())
#     # Maybe remove this pooling layer if the size reduces too much
#     # model.add(MaxPool1D(pool_size=2, strides=2))
#     model.add(Dropout(0.5))

#     model.add(Flatten())

#     # Dense layers
#     model.add(Dense(256))
#     model.add(Dense(128))
#     model.add(Dense(64))
#     model.add(Dense(32))

#     model.add(Dense(1, activation='sigmoid'))  # Output layer

#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
#     return model

# # Create the model
# model = cnnmodel(input_shape=(29, 33))
# model.summary()


In [ ]:

# model.fit(scale_3d_data(X_train), Y_train, epochs=50, batch_size=32, validation_data=(scale_3d_data(X_test), Y_test))

In [ ]:
clear_session()
model = Sequential()
# Encoder network
model.add(Conv1D(35, 3, activation='relu', padding='same', input_shape=(29, 180)))
model.add(MaxPool1D(2, padding='same'))
model.add(Conv1D(25, 3, activation='relu', padding='same'))
model.add(MaxPool1D(2, padding='same'))

# Decoder network
model.add(Conv1D(25, 3, activation='relu', padding='same'))
model.add(UpSampling1D(2))
model.add(Conv1D(35, 3, activation='relu', padding='same'))
model.add(UpSampling1D(2))

# Output layer with a Reshape layer to match the input shape
model.add(Conv1D(180, 3, activation='sigmoid', padding='same'))  # Conv1D layer
# model.add(Reshape((29, 180)))  # Reshape to match input shape

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Using mean squared error for reconstruction

# Fit the model
model.fit(X_train_Scale, X_train_Scale,  # Target is the input itself
          epochs=15,
          batch_size=128,
          validation_data=(X_test_Scale, X_test_Scale))

Epoch 1/15


ValueError: in user code:

    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\

In [ ]:
X_train_Scale.shape,X_test_Scale.shape

((17124, 29, 180), (4282, 29, 180))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, UpSampling1D

clear_session()
model = Sequential()
# Encoder network
model.add(Conv1D(35, 3, activation='relu', padding='same', input_shape=(29, 180)))
model.add(MaxPool1D(2, padding='valid'))  # 'valid' padding will reduce the dimension
model.add(Conv1D(25, 3, activation='relu', padding='same'))
model.add(MaxPool1D(2, padding='valid'))  # Further reduction

# Decoder network
model.add(Conv1D(25, 3, activation='relu', padding='same'))
model.add(UpSampling1D(2))  # Upsampling to increase the dimension
model.add(Conv1D(35, 3, activation='relu', padding='same'))
model.add(UpSampling1D(2))  # Further increasing to match the original dimension

# Output layer - should match the input dimensions
model.add(Conv1D(180, 3, activation='sigmoid', padding='same'))  # Final layer to match the input shape

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Using mean squared error for reconstruction

# Model Summary
model.summary()

# Fit the model
model.fit(X_train_Scale, X_train_Scale,  # Target is the input itself
          epochs=15,
          batch_size=128,
          validation_data=(X_test_Scale, X_test_Scale))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 29, 35)            18935     
                                                                 
 max_pooling1d (MaxPooling1  (None, 14, 35)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 25)            2650      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 7, 25)             0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 7, 25)             1900      
                                                                 
 up_sampling1d (UpSampling1  (None, 14, 25)            0

ValueError: in user code:

    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Dense, Flatten, Reshape, LeakyReLU


model = Sequential()

# Encoder
model.add(Conv1D(filters=35, kernel_size=3, activation='relu', padding='same', input_shape=(29,180)))
model.add(MaxPooling1D(pool_size=3, padding='same'))
model.add(Conv1D(filters=25, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))# Decoder
model.add(Conv1D(filters=25, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(2))
model.add(Conv1D(filters=35, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(2))

# Output layer
model.add(Conv1D(filters=180, kernel_size=3, activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error')
# Model Summary
model.summary()

# Fit the model
model.fit(X_train_Scale, X_train_Scale,  # Target is the input itself
          epochs=15,
          batch_size=128,
          validation_data=(X_test_Scale, X_test_Scale))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          (None, 29, 35)            18935     
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 10, 35)            0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 10, 25)            2650      
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 5, 25)             0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 5, 25)             1900      
                                                                 
 up_sampling1d_6 (UpSamplin  (None, 10, 25)           

ValueError: in user code:

    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\

# Equalized data

In [ ]:
# # Path to the file
# file_path3 = 'E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/time_domaindata.npz'
# file_path4 = 'E:/Data WareHouse/1sec/Epileptic Seizure/feature extracted data/feature_domaindata.npz'
# # Load the array
# time_domain_features = np.load(file_path3)
# frequencydomain = np.load(file_path4)


In [ ]:

# X = time_domain_features['X']
# Y = time_domain_features['Y']
# # group1 = npz_file['G']

In [ ]:
# Eq_X.shape

(946, 29, 180)

In [ ]:


# Assuming all_features and frequency_features are your numpy arrays
delta_merged_features = np.concatenate((delta_time_domain_features ,delta_frequency_features), axis=2)
theta_merged_features = np.concatenate((theta_time_domain_features ,theta_frequency_features), axis=2)
alpha_merged_features = np.concatenate((alpha_time_domain_features ,alpha_frequency_features), axis=2)
beta_merged_features = np.concatenate((beta_time_domain_features ,beta_frequency_features), axis=2)

# Checking the new shape
# print(merged_features.shape)

In [ ]:
beta_frequency_features.shape,beta_time_domain_features.shape

((5834, 29, 20), (5834, 29, 13))

In [ ]:
#Change the shape of the array
# Reshape the data to 2D
delta_feature_reshaped = delta_merged_features.reshape(delta_merged_features.shape[0], -1)
theta_feature_reshaped = theta_merged_features.reshape(theta_merged_features.shape[0], -1)
alpha_feature_reshaped = alpha_merged_features.reshape(alpha_merged_features.shape[0], -1)
beta_feature_reshaped = beta_merged_features.reshape(beta_merged_features.shape[0], -1)

In [ ]:
beta_merged_features.shape

NameError: name 'beta_merged_features' is not defined

##merge data

In [ ]:
t_f_domain_folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW'
delta_path, theta_path, alpha_path, beta_path = [os.path.join(t_f_domain_folder,i) for i in os.listdir(t_f_domain_folder)]

In [ ]:
delta_path

'/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW/delta_MERGE_domain.npy'

In [ ]:
delta_data= np.load(delta_path)
theta_data= np.load(theta_path)
alpha_data= np.load(alpha_path)
beta_data= np.load(beta_path)

In [ ]:
delta_reshape = delta_data.reshape(delta_data.shape[0], -1)
theta_reshape = theta_data.reshape(theta_data.shape[0], -1)
alpha_reshape = alpha_data.reshape(alpha_data.shape[0], -1)
beta_reshape = beta_data.reshape(beta_data.shape[0], -1)

In [ ]:
# time_D_features_train = merged_features.reshape(time_domain_features.shape[0], -1)

In [ ]:
# frequency_D_features_train =  merged_features.reshape(frequencydomain.shape[0], -1)

In [ ]:
delta_feature_reshaped.shape, theta_feature_reshaped.shape, alpha_feature_reshaped.shape, beta_feature_reshaped.shape

((5834, 957), (5834, 957), (5834, 957), (5834, 957))

In [ ]:
# X1,X0,Y1,Y0 = data_sep(delta_feature_reshaped,Y)

In [ ]:
# X_train,Y_train, X_test, Y_test = train_test_spilt(X1,X0,Y1,Y0,split=.3)

In [ ]:
# X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4084, 377), (1750, 377), (4084,), (1750,))

In [ ]:
# X_train[Y_train==1].shape,X_test[Y_test==1].shape

((2042, 377), (875, 377))

In [ ]:
# X_train[Y_train==0].shape,X_test[Y_test==0].shape

((2042, 377), (875, 377))

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,\
                                StackingClassifier,VotingClassifier,IsolationForest,\
                                RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
# import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier

from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV,GridSearchCV

In [ ]:
#Change the shape of the array
# Reshape the data to 2D
# eeg_frequency_feature = merged_features.reshape(merged_features.shape[0], -1)

In [ ]:
#Scale the data
SS1= StandardScaler()
delta_X = SS1.fit_transform(delta_reshape)
SS2= StandardScaler()
theta_X = SS2.fit_transform(theta_reshape)
SS3= StandardScaler()
alpha_X = SS3.fit_transform(alpha_reshape)
SS4= StandardScaler()
beta_X = SS4.fit_transform(beta_reshape)

In [ ]:
# #Scale the data
# SS= StandardScaler()
# X_train_Ss = SS.fit_transform(X_train)

In [ ]:
# #Scale the data
# SS= StandardScaler()
# X_test_Ss = SS.fit_transform(X_test)

In [ ]:
models = {
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),
    # 'GaussianProcess': GaussianProcessClassifier(kernel=RBF()),
    # 'CategoricalNB': CategoricalNB(),
    # 'LightGBM': lgb.LGBMClassifier(),
    # 'RandomForestRegressor': RandomForestRegressor(),  # Note: This is a regressor, not a classifier
    # 'IsolationForest': IsolationForest()  # Note: This is typically used for anomaly detection
    # Add any additional models here
}

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef
from sklearn.model_selection import KFold
import pandas as pd

def evaluate_model(model, X, y, n_splits):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=42)
    accuracies, precisions, recalls,f1s,cm,specificitys, sensitivitys,roc_aucs,kappas,mccs= [], [], [],[],[],[],[],[],[],[]

    for train_idx, test_idx in kf.split(X):
      # take the location of the splited data and access it by the index

      X_train, y_train = (X.iloc[train_idx], y.iloc[train_idx]) if isinstance(X, pd.DataFrame) else (X[train_idx], y[train_idx])
      X_test, y_test = (X.iloc[test_idx], y.iloc[test_idx]) if isinstance(X, pd.DataFrame) else (X[test_idx], y[test_idx])
      # train the model
      model.fit(X_train, y_train)
      # make prediction
      y_pred = model.predict(X_test)

      # evaluation scores
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred,zero_division=1)
      recall = recall_score(y_test, y_pred)
      f1= f1_score(y_test, y_pred)
      cms = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix
      roc_auc=roc_auc_score(y_test, y_pred)

      # calculate the specificity and sensitivity
      tn, fp, fn, tp = cms.ravel()
      specificity = tn / (tn + fp)
      sensitivity = tp / (tp + fn)
      # Calculate Cohen's Kappa
      kappa = cohen_kappa_score(y_test, y_pred)
      mcc = matthews_corrcoef(y_test, y_pred)





      # append all the results
      accuracies.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1s.append(f1)
      cm.append(cms)  # Add confusion matrix to list
      specificitys.append(specificity) #add specificity score
      sensitivitys.append(sensitivity) #add sensitivity scores to the list
      roc_aucs.append(roc_auc) #add roc_auc score to the list
      kappas.append(kappa)
      mccs.append(mcc)


    return (format(sum(accuracies)/n_splits, '.3f'), format(sum(precisions)/n_splits, '.3f'),
            format(sum(recalls)/n_splits, '.3f'),format(sum(f1s)/n_splits, '.3f'),format(sum(roc_aucs)/n_splits, '.3f'), sum(cm)/n_splits ,format(sum(specificitys)/n_splits, '.3f'),
            format(sum(sensitivitys)/n_splits, '.3f'),format(sum(kappas)/n_splits, '.3f'),format(sum(mccs)/n_splits, '.3f'))




def modelApplied(Model,X_train , y_train, n_splits=5,MN='model_name',model_result = pd.DataFrame()):
  accuracy, precision, recall,f1,roc_auc,cm,specificity,sensitivity,kappa,mcc = evaluate_model(Model,X_train , y_train, n_splits)

  new_row=[MN,accuracy,precision,recall,f1,roc_auc,specificity, sensitivity,kappa,mcc]

  model_result.loc[len(model_result)]=new_row
  print(new_row)
  print('DONE!!')





In [ ]:
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Delta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
delta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, delta_X, Y, MN=k,model_result = delta_result)



theta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])


print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>theta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, theta_X, Y, MN=k,model_result= theta_result)


alpha_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>alpha<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, alpha_X, Y, MN=k,model_result= alpha_result)



beta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])



print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Beta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, beta_X, Y, MN=k,model_result= beta_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Delta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.913', '0.963', '0.858', '0.908', '0.913', '0.967', '0.858', '0.826', '0.831']
DONE!!
['RandomForest', '0.906', '0.943', '0.864', '0.902', '0.906', '0.948', '0.864', '0.812', '0.815']
DONE!!
['HistGradientBoosting', '0.910', '0.961', '0.856', '0.905', '0.910', '0.965', '0.856', '0.821', '0.826']
DONE!!
['SVM', '0.869', '0.931', '0.796', '0.858', '0.869', '0.941', '0.796', '0.737', '0.745']
DONE!!
['GradientBoosting', '0.897', '0.937', '0.852', '0.892', '0.897', '0.942', '0.852', '0.795', '0.798']
DONE!!
['KNeighbors', '0.603', '0.873', '0.242', '0.377', '0.603', '0.965', '0.242', '0.207', '0.299']
DONE!!
['MLP', '0.877', '0.906', '0.841', '0.872', '0.877', '0.913', '0.841', '0.754', '0.756']
DONE!!
['DecisionTree', '0.841', '0.843', '0.837', '0.840', '0.841', '0.844', '0.837', '0.681', '0.682']
DONE!!
['GaussianNB', '0.850', '0.976', '0.717', '0.826', '0.849', '0.982', '0.717', '0.699',

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.859', '0.882', '0.830', '0.855', '0.860', '0.889', '0.830', '0.719', '0.720']
DONE!!
['AdaBoost', '0.874', '0.905', '0.836', '0.869', '0.874', '0.913', '0.836', '0.748', '0.751']
DONE!!
['ExtraTrees', '0.908', '0.947', '0.864', '0.904', '0.908', '0.952', '0.864', '0.816', '0.819']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>theta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.905', '0.948', '0.858', '0.900', '0.905', '0.952', '0.858', '0.810', '0.814']
DONE!!
['RandomForest', '0.900', '0.937', '0.857', '0.895', '0.899', '0.942', '0.857', '0.799', '0.802']
DONE!!
['HistGradientBoosting', '0.909', '0.959', '0.854', '0.903', '0.909', '0.964', '0.854', '0.818', '0.823']
DONE!!
['SVM', '0.864', '0.910', '0.807', '0.855', '0.864', '0.920', '0.807', '0.727', '0.732']
DONE!!
['GradientBoosting', '0.886', '0.918', '0.849', '0.882', '0.886', '0.924', '0.849', '0.773', '0.775']
DONE!!
['KNeighbors', '0.667', '0.850', '0.408', '0.549', '0.667', '0.927', '0.408

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.852', '0.879', '0.816', '0.846', '0.852', '0.888', '0.816', '0.704', '0.706']
DONE!!
['AdaBoost', '0.869', '0.899', '0.832', '0.864', '0.869', '0.906', '0.832', '0.738', '0.741']
DONE!!
['ExtraTrees', '0.892', '0.923', '0.855', '0.888', '0.892', '0.929', '0.855', '0.784', '0.786']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>alpha<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.872', '0.899', '0.839', '0.867', '0.872', '0.905', '0.839', '0.744', '0.746']
DONE!!
['RandomForest', '0.858', '0.867', '0.846', '0.856', '0.858', '0.870', '0.846', '0.716', '0.716']
DONE!!
['HistGradientBoosting', '0.874', '0.903', '0.837', '0.869', '0.874', '0.910', '0.837', '0.747', '0.749']
DONE!!
['SVM', '0.806', '0.850', '0.745', '0.794', '0.806', '0.868', '0.745', '0.613', '0.618']
DONE!!
['GradientBoosting', '0.850', '0.862', '0.832', '0.847', '0.850', '0.867', '0.832', '0.699', '0.700']
DONE!!
['KNeighbors', '0.601', '0.784', '0.281', '0.412', '0.601', '0.921', '0.281

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.790', '0.813', '0.754', '0.782', '0.790', '0.826', '0.754', '0.580', '0.581']
DONE!!
['AdaBoost', '0.804', '0.823', '0.774', '0.798', '0.804', '0.834', '0.774', '0.608', '0.609']
DONE!!
['ExtraTrees', '0.856', '0.859', '0.852', '0.855', '0.856', '0.860', '0.852', '0.712', '0.712']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Beta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.920', '0.943', '0.895', '0.918', '0.920', '0.945', '0.895', '0.841', '0.842']
DONE!!
['RandomForest', '0.897', '0.914', '0.878', '0.895', '0.898', '0.917', '0.878', '0.795', '0.796']
DONE!!
['HistGradientBoosting', '0.921', '0.946', '0.892', '0.918', '0.921', '0.949', '0.892', '0.842', '0.843']
DONE!!
['SVM', '0.872', '0.900', '0.838', '0.867', '0.872', '0.906', '0.838', '0.744', '0.746']
DONE!!
['GradientBoosting', '0.899', '0.921', '0.874', '0.897', '0.899', '0.925', '0.874', '0.799', '0.800']
DONE!!
['KNeighbors', '0.673', '0.876', '0.404', '0.551', '0.673', '0.941', '0.404'

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.863', '0.878', '0.844', '0.860', '0.863', '0.883', '0.844', '0.726', '0.727']
DONE!!
['AdaBoost', '0.877', '0.889', '0.862', '0.875', '0.877', '0.893', '0.862', '0.754', '0.755']
DONE!!
['ExtraTrees', '0.897', '0.917', '0.873', '0.894', '0.897', '0.921', '0.873', '0.794', '0.795']
DONE!!


In [ ]:
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Delta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
delta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, delta_X, Y, MN=k,model_result = delta_result)



theta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])


print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>theta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, theta_X, Y, MN=k,model_result= theta_result)


alpha_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>alpha<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, alpha_X, Y, MN=k,model_result= alpha_result)



beta_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])



print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Beta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# Assuming Models is your dictionary of models
for k, v in models.items():
    modelApplied(v, beta_X, Y, MN=k,model_result= beta_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Delta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.909', '0.959', '0.855', '0.904', '0.909', '0.963', '0.855', '0.819', '0.824']
DONE!!
['RandomForest', '0.909', '0.948', '0.865', '0.904', '0.909', '0.952', '0.865', '0.817', '0.820']
DONE!!
['HistGradientBoosting', '0.913', '0.965', '0.858', '0.908', '0.913', '0.969', '0.858', '0.827', '0.832']
DONE!!
['SVM', '0.865', '0.947', '0.772', '0.851', '0.865', '0.957', '0.772', '0.729', '0.742']
DONE!!
['GradientBoosting', '0.899', '0.938', '0.853', '0.894', '0.898', '0.944', '0.853', '0.797', '0.800']
DONE!!
['KNeighbors', '0.610', '0.856', '0.264', '0.404', '0.610', '0.956', '0.264', '0.220', '0.304']
DONE!!
['MLP', '0.873', '0.908', '0.829', '0.867', '0.873', '0.916', '0.829', '0.745', '0.748']
DONE!!
['DecisionTree', '0.829', '0.827', '0.832', '0.829', '0.829', '0.825', '0.832', '0.658', '0.658']
DONE!!
['GaussianNB', '0.832', '0.978', '0.679', '0.801', '0.832', '0.985', '0.679', '0.664',

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.856', '0.879', '0.825', '0.851', '0.856', '0.886', '0.825', '0.711', '0.713']
DONE!!
['AdaBoost', '0.868', '0.898', '0.830', '0.863', '0.868', '0.905', '0.830', '0.736', '0.738']
DONE!!
['Bagging', '0.890', '0.934', '0.839', '0.884', '0.890', '0.941', '0.839', '0.780', '0.784']
DONE!!
['ExtraTrees', '0.904', '0.942', '0.861', '0.899', '0.904', '0.947', '0.861', '0.807', '0.810']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>theta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.904', '0.945', '0.858', '0.899', '0.904', '0.950', '0.858', '0.807', '0.811']
DONE!!
['RandomForest', '0.899', '0.933', '0.860', '0.895', '0.899', '0.938', '0.860', '0.799', '0.802']
DONE!!
['HistGradientBoosting', '0.907', '0.954', '0.855', '0.902', '0.907', '0.959', '0.855', '0.814', '0.819']
DONE!!
['SVM', '0.866', '0.928', '0.793', '0.855', '0.866', '0.939', '0.793', '0.732', '0.740']
DONE!!
['GradientBoosting', '0.890', '0.927', '0.846', '0.885', '0.890', '0.933', '0.846', 

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.845', '0.866', '0.815', '0.840', '0.845', '0.874', '0.815', '0.690', '0.691']
DONE!!
['AdaBoost', '0.863', '0.896', '0.821', '0.857', '0.863', '0.905', '0.821', '0.726', '0.729']
DONE!!
['Bagging', '0.880', '0.918', '0.835', '0.874', '0.880', '0.926', '0.835', '0.760', '0.764']
DONE!!
['ExtraTrees', '0.890', '0.915', '0.861', '0.887', '0.890', '0.919', '0.861', '0.780', '0.782']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>alpha<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.860', '0.894', '0.817', '0.854', '0.860', '0.903', '0.817', '0.720', '0.723']
DONE!!
['RandomForest', '0.848', '0.862', '0.827', '0.844', '0.848', '0.868', '0.827', '0.695', '0.696']
DONE!!
['HistGradientBoosting', '0.861', '0.896', '0.817', '0.855', '0.861', '0.905', '0.817', '0.722', '0.725']
DONE!!
['SVM', '0.787', '0.839', '0.712', '0.770', '0.787', '0.863', '0.712', '0.575', '0.582']
DONE!!
['GradientBoosting', '0.837', '0.848', '0.822', '0.834', '0.837', '0.852', '0.822', 

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.770', '0.788', '0.738', '0.762', '0.770', '0.802', '0.738', '0.540', '0.541']
DONE!!
['AdaBoost', '0.793', '0.816', '0.757', '0.785', '0.793', '0.829', '0.757', '0.586', '0.588']
DONE!!
['Bagging', '0.828', '0.861', '0.782', '0.820', '0.828', '0.874', '0.782', '0.656', '0.659']
DONE!!
['ExtraTrees', '0.849', '0.859', '0.836', '0.847', '0.849', '0.863', '0.836', '0.699', '0.699']
DONE!!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Beta<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['XGBoost', '0.887', '0.906', '0.863', '0.884', '0.887', '0.911', '0.863', '0.773', '0.774']
DONE!!
['RandomForest', '0.875', '0.885', '0.861', '0.873', '0.875', '0.889', '0.861', '0.749', '0.750']
DONE!!
['HistGradientBoosting', '0.892', '0.914', '0.865', '0.889', '0.892', '0.919', '0.865', '0.784', '0.785']
DONE!!
['SVM', '0.780', '0.832', '0.703', '0.762', '0.781', '0.858', '0.703', '0.561', '0.568']
DONE!!
['GradientBoosting', '0.871', '0.885', '0.852', '0.868', '0.871', '0.889', '0.852', '

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.798', '0.818', '0.767', '0.791', '0.798', '0.829', '0.767', '0.596', '0.597']
DONE!!
['AdaBoost', '0.831', '0.849', '0.806', '0.827', '0.831', '0.857', '0.806', '0.662', '0.664']
DONE!!
['Bagging', '0.853', '0.880', '0.818', '0.848', '0.853', '0.889', '0.818', '0.707', '0.708']
DONE!!
['ExtraTrees', '0.870', '0.883', '0.853', '0.868', '0.870', '0.887', '0.853', '0.741', '0.741']
DONE!!


In [ ]:
# Final_Results.to_csv('E:/Data WareHouse/1sec/Epileptic Seizure/Results/time_feature_domain_results_ml.csv', index= False)

In [ ]:
delta_result.to_csv('',index=False)

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Kappa,MCC
0,XGBoost,0.909,0.959,0.855,0.904,0.909,0.963,0.855,0.819,0.824
1,RandomForest,0.909,0.948,0.865,0.904,0.909,0.952,0.865,0.817,0.820
2,HistGradientBoosting,0.913,0.965,0.858,0.908,0.913,0.969,0.858,0.827,0.832
3,SVM,0.865,0.947,0.772,0.851,0.865,0.957,0.772,0.729,0.742
4,GradientBoosting,0.899,0.938,0.853,0.894,0.898,0.944,0.853,0.797,0.800
5,KNeighbors,0.610,0.856,0.264,0.404,0.610,0.956,0.264,0.220,0.304
6,MLP,0.873,0.908,0.829,0.867,0.873,0.916,0.829,0.745,0.748
7,DecisionTree,0.829,0.827,0.832,0.829,0.829,0.825,0.832,0.658,0.658
8,GaussianNB,0.832,0.978,0.679,0.801,0.832,0.985,0.679,0.664,0.697
9,BernoulliNB,0.833,0.968,0.688,0.804,0.833,0.978,0.688,0.666,0.696


In [ ]:
theta_result

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Kappa,MCC
0,XGBoost,0.904,0.945,0.858,0.899,0.904,0.950,0.858,0.807,0.811
1,RandomForest,0.899,0.933,0.860,0.895,0.899,0.938,0.860,0.799,0.802
2,HistGradientBoosting,0.907,0.954,0.855,0.902,0.907,0.959,0.855,0.814,0.819
3,SVM,0.866,0.928,0.793,0.855,0.866,0.939,0.793,0.732,0.740
4,GradientBoosting,0.890,0.927,0.846,0.885,0.890,0.933,0.846,0.779,0.783
5,KNeighbors,0.627,0.831,0.320,0.461,0.627,0.933,0.320,0.253,0.322
6,MLP,0.868,0.902,0.826,0.862,0.868,0.911,0.826,0.737,0.739
7,DecisionTree,0.820,0.818,0.823,0.820,0.820,0.816,0.823,0.639,0.640
8,GaussianNB,0.772,0.964,0.565,0.712,0.772,0.978,0.565,0.543,0.597
9,BernoulliNB,0.801,0.930,0.650,0.765,0.801,0.951,0.650,0.601,0.631


In [ ]:
alpha_result

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Kappa,MCC
0,XGBoost,0.860,0.894,0.817,0.854,0.860,0.903,0.817,0.720,0.723
1,RandomForest,0.848,0.862,0.827,0.844,0.848,0.868,0.827,0.695,0.696
2,HistGradientBoosting,0.861,0.896,0.817,0.855,0.861,0.905,0.817,0.722,0.725
3,SVM,0.787,0.839,0.712,0.770,0.787,0.863,0.712,0.575,0.582
4,GradientBoosting,0.837,0.848,0.822,0.834,0.837,0.852,0.822,0.674,0.674
5,KNeighbors,0.607,0.680,0.406,0.508,0.607,0.808,0.406,0.215,0.235
6,MLP,0.806,0.828,0.771,0.799,0.806,0.840,0.771,0.611,0.613
7,DecisionTree,0.773,0.777,0.766,0.771,0.773,0.780,0.766,0.546,0.546
8,GaussianNB,0.686,0.944,0.396,0.558,0.686,0.977,0.396,0.373,0.458
9,BernoulliNB,0.708,0.866,0.493,0.628,0.709,0.924,0.493,0.417,0.462


In [ ]:
beta_result

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Kappa,MCC
0,XGBoost,0.887,0.906,0.863,0.884,0.887,0.911,0.863,0.773,0.774
1,RandomForest,0.875,0.885,0.861,0.873,0.875,0.889,0.861,0.749,0.750
2,HistGradientBoosting,0.892,0.914,0.865,0.889,0.892,0.919,0.865,0.784,0.785
3,SVM,0.780,0.832,0.703,0.762,0.781,0.858,0.703,0.561,0.568
4,GradientBoosting,0.871,0.885,0.852,0.868,0.871,0.889,0.852,0.741,0.742
5,KNeighbors,0.580,0.749,0.242,0.365,0.580,0.919,0.242,0.161,0.218
6,MLP,0.800,0.818,0.773,0.794,0.800,0.827,0.773,0.600,0.602
7,DecisionTree,0.792,0.798,0.782,0.790,0.792,0.802,0.782,0.584,0.584
8,GaussianNB,0.640,0.932,0.302,0.456,0.640,0.978,0.302,0.280,0.380
9,BernoulliNB,0.673,0.859,0.414,0.558,0.673,0.932,0.414,0.345,0.404


In [ ]:
RF=RandomForestClassifier()
ada=AdaBoostClassifier(base_estimator= RandomForestClassifier())
RFC=Pipeline([('scaler',StandardScaler()),('RF',RF)])
# ABC=Pipeline([('scaler',StandardScaler()),('ada',ada)])
# # param_grid = {
#     'RF__n_estimators': [100],  # Try different values for n_estimators
#     # 'RF__max_depth': [],  # Try different values for max_depth
# }

# model=GridSearchCV(estimator=pipe,param_grid=param_grid, cv=5)

In [ ]:
param_grid = {
    'RF__n_estimators': [611],  # Try different values for n_estimators
    # 'RF__criterion': ['entropy'],
    # 'RF__max_depth' : [7],

    # 'RF__min_samples_split': np.arange(1,10)

    # 'RF__max_depth': [],  # Try different values for max_depth
}

model=GridSearchCV(estimator=RFC,param_grid=param_grid, cv=5)

In [ ]:
# Random Forest
model.fit(X_train, Y_train)
print(model.best_estimator_)
print(model.best_score_)

C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
5 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sk

Pipeline(steps=[('scaler', StandardScaler()),
                ('RF',
                 RandomForestClassifier(criterion='entropy', max_depth=7,
                                        min_samples_split=9,
                                        n_estimators=611))])
0.8655734754121968


In [ ]:
# Random Forest
RFC.fit(X_train, Y_train)

pred = RFC.predict(X_test)

accuracy_score(Y_test, pred)

In [ ]:
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

Precision: 0.87
Recall: 0.68
F1 Score: 0.76
Sensitivity: 0.68
Specificity: 0.89


In [ ]:
# Random Forest
ABC.fit(X_train, Y_train)

pred = ABC.predict(X_test)

accuracy_score(Y_test, pred)

C:\Users\Administrator\anaconda3\envs\mne\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.7789473684210526

In [ ]:
precision = precision_score(Y_test, pred)
recall = recall_score(Y_test, pred)
f1 = f1_score(Y_test, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(Y_test, pred).ravel()

# Sensitivity (True Positive Rate) and Specificity (True Negative Rate)
sensitivity = tp / (tp + fn)  # or use 'recall'
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

Precision: 0.85
Recall: 0.67
F1 Score: 0.75
Sensitivity: 0.67
Specificity: 0.88


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

def evaluate_model(model, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'cm': [], 'specificity': [], 'sensitivity': []}

    for train_idx, test_idx in kf.split(X):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        results['accuracy'].append(accuracy_score(y_test, y_pred))
        results['precision'].append(precision_score(y_test, y_pred, zero_division=1))
        results['recall'].append(recall_score(y_test, y_pred))
        results['f1'].append(f1_score(y_test, y_pred))
        results['roc_auc'].append(roc_auc_score(y_test, y_pred))
        results['specificity'].append(tn / (tn + fp))
        results['sensitivity'].append(tp / (tp + fn))
        results['cm'].append(confusion_matrix(y_test, y_pred))

    return {metric: np.mean(values) for metric, values in results.items()}

def evaluate_models(models, X, y, n_splits=5):
    final_results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC_AUC', 'Specificity', 'Sensitivity', 'Confusion Matrix'])

    for model_name, model in models.items():
        result = evaluate_model(model, X, y, n_splits)
        result_row = {
            'Model': model_name,
            'Accuracy': result['accuracy'],
            'Precision': result['precision'],
            'Recall': result['recall'],
            'F1-Score': result['f1'],
            'ROC_AUC': result['roc_auc'],
            'Specificity': result['specificity'],
            'Sensitivity': result['sensitivity'],
            'Confusion Matrix': result['cm']
        }
        final_results = pd.concat([final_results, pd.DataFrame([result_row])], ignore_index=True)

    return final_results

# Define your models here
models = {
    'RandomForest': RandomForestClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'Bagging': BaggingClassifier(),
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'XGBoost': XGBClassifier()
}

# Assuming X_train and Y_train are your training data and labels
final_results = evaluate_models(models, frequency_D_features_train, Eq_Y)
print(final_results)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8824\1795261679.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_results = pd.concat([final_results, pd.DataFrame([result_row])], ignore_index=True)


                  Model  Accuracy  Precision    Recall  F1-Score   ROC_AUC  \
0          RandomForest  0.762150   0.782202  0.729503  0.753505  0.764035   
1              AdaBoost  0.697639   0.705368  0.684595  0.693002  0.699744   
2            ExtraTrees  0.767446   0.791048  0.728511  0.757480  0.768385   
3      GradientBoosting  0.756887   0.774039  0.728367  0.748939  0.758594   
4               Bagging  0.744194   0.765458  0.702264  0.732144  0.744159   
5  HistGradientBoosting  0.766360   0.804692  0.701770  0.748706  0.766707   
6          DecisionTree  0.665987   0.665722  0.662714  0.663613  0.665863   
7               XGBoost  0.760006   0.800289  0.696556  0.742230  0.761287   

   Specificity  Sensitivity  Confusion Matrix  
0     0.798567     0.729503              47.3  
1     0.714893     0.684595              47.3  
2     0.808258     0.728511              47.3  
3     0.788821     0.728367              47.3  
4     0.786054     0.702264              47.3  
5     0.8

In [ ]:
final_results


,Model,Accuracy,Precision,Recall,F1-Score,ROC_AUC,Specificity,Sensitivity,Confusion Matrix
0,RandomForest,0.762150,0.782202,0.729503,0.753505,0.764035,0.798567,0.729503,47.3
1,AdaBoost,0.697639,0.705368,0.684595,0.693002,0.699744,0.714893,0.684595,47.3
2,ExtraTrees,0.767446,0.791048,0.728511,0.757480,0.768385,0.808258,0.728511,47.3
3,GradientBoosting,0.756887,0.774039,0.728367,0.748939,0.758594,0.788821,0.728367,47.3
4,Bagging,0.744194,0.765458,0.702264,0.732144,0.744159,0.786054,0.702264,47.3
5,HistGradientBoosting,0.766360,0.804692,0.701770,0.748706,0.766707,0.831645,0.701770,47.3
6,DecisionTree,0.665987,0.665722,0.662714,0.663613,0.665863,0.669013,0.662714,47.3
7,XGBoost,0.760006,0.800289,0.696556,0.742230,0.761287,0.826018,0.696556,47.3


In [ ]:
final_results.to_csv("E:/Data WareHouse/result/frequency_domain_ng.csv")